In [52]:
import asyncio
import re
import os
import pandas as pd
import base64
import io
from PIL import Image
import base64
from io import BytesIO
from openai import AsyncOpenAI
from dotenv import load_dotenv
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, stop_after_delay,
)

# Load environment variables
load_dotenv()

system_prompt_expansion='''According to the user query, expand and solidify the query into a step by step detailed instruction (or comment) on how to write python code to fulfill the user query's requirements. Import the appropriate libraries. Pinpoint the correct library functions to call and set each parameter in every function call accordingly.'''

template_expansion ="""
            Generate three distinct extended queries based on the given original query.  
            Each extended query should be written in a Chain of Thought (CoT) style, explicitly outlining the reasoning and step-by-step methodology to achieve the goal.  

            ### **Key Requirements:**  
            - Do NOT change the goal or instructions given in the original query.  
            - Propose **three different methodologies** to achieve the goal while maintaining the original intent.  
            - Each extended query must be structured in a step-by-step CoT format, explaining **why** each step is necessary.  
            - The different methodologies should vary in terms of **data processing, visualization techniques, or computation strategies**.  
            - Ensure that the data description is analyzed and incorporated into the query design.  
            - If no data description is provided, then strictly follow the original query.  

            ### **Variations in Approach (Examples):**  
            - Using different **data processing techniques** (e.g., pandas, NumPy, direct iteration)  
            - Implementing various **visualization strategies** (e.g., different libraries, different styles of plots)  
            - Exploring alternative **computation methods** (e.g., vectorized operations, grouped aggregations, iterative filtering)  

            ### **Input:**  
            The original query is: {ori_query}  

            Data description is: {data_description}  

            ### **Output Format:**  
            Return the output **ONLY** in the following Python list format:  
            ```[query_text_1, query_text_2, query_text_3]```  
            """
        
system_prompt_codegen="""You are an expert on data visualization code generation. You should think step by step, and write the generated code in the format of ```python...```, where ... indicates the generated code. Code Must end in plt.show() and don't save figure by plt.savefig() and don't save anything else either."""
        
template_codegen="""\
            Based on the user's query and data description, generate Python code using Visualization Library like `matplotlib.pyplot` or 'seaborn' to create the requested plot. Ensure the code is outputted within the format ```...```, where ... indicates the generated code. Please make sure to generate the code according to the data description below. Do not include unnecessary code for saving plot figure, saving dataframe, or other unrelated tasks when generating the code. End with plt.show() and do not include anything after that.
            You Must use data_path provided in Data Description when loading data with pd.read_csv().

            User query: {query}

            Data description: {data_description}
            """

system_prompt_aggregation="""
        You are an expert in analyzing, improving, and synthesizing data visualization code. 
        Your role is to evaluate multiple versions of visualization code based on user queries and data descriptions, 
        integrate feedback effectively, and generate a final version that best meets the user's requirements."
"""

template_aggregation="""\
    Think step by step and plan before generating the final code.

You will be given:
- **User Query**: Instructions on how the user wants the data visualization (plot) to be performed.
- **Data Description**: Details about the dataset, including file paths and summaries.
- **Code for Aggregation with Corresponding Feedback**: Three different versions of the data visualization code, each paired with its respective feedback highlighting mismatches with the user’s requirements and areas for improvement.

### **Your Task:**
1. **Understand the User's Intent**  
   - Analyze the **User Query** to extract key visualization requirements, constraints, and goals.
   - Carefully review the **Data Description** to ensure the final visualization correctly utilizes the given dataset.

2. **Evaluate the Provided Code Versions & Feedback**  
   - Examine all three versions of the code.
   - Review the feedback for each version and identify common issues, missing elements, and improvement points.
   - Determine which parts of each version align well with the user's requirements.

3. **Synthesize the Best Final Version**  
   - Construct a final version that effectively **integrates the best aspects** of the provided codes while addressing all necessary corrections from the feedback.  
   - Ensure the final code adheres to **high readability, clarity, and maintainability** while fully complying with the user’s original instructions.  
   - Eliminate unnecessary complexity while maintaining functionality.

4. **Output the Final Version**  
   - Provide the optimized final version inside a properly formatted code block:
     ```
     ```python
     # Final optimized code
     ...
     ```
     ```

### **Inputs:**
- **User Query:** {ori_query}
- **Data Description:** {data_description}
- **Code for Aggregation with Corresponding Feedback:** {code_for_aggregation}
"""

In [53]:
import tiktoken

def count_gpt4o_tokens(text: str) -> int:
    """
    GPT-4o 모델의 입력 텍스트에 대한 토큰 개수를 계산하는 함수
    """
    encoding = tiktoken.get_encoding("cl100k_base")  # GPT-4o에서 사용하는 인코딩
    tokens = encoding.encode(text)
    print(f"Token count: {len(tokens)}")
    return len(tokens)

# 테스트 예제
sample_text = "Hello, how are you?"
token_count = count_gpt4o_tokens(sample_text)

Token count: 6


In [54]:
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")

client = AsyncOpenAI(
    api_key=api_key,
    base_url=base_url
)

model = 'gpt-4o-mini'
temperature = 0.2
# sample = [10, 20, 30, 40, 50]
sample = 40
dataset_path = r"..\..\dataset\matplotbench_data.csv"

dataset_df = pd.read_csv(dataset_path)
query_list = dataset_df.loc[sample,'simple_instruction']

In [55]:
img_save_path = f'./{sample}.png'
data_description='''There is no dataset provided.'''
# QUERY_EXPANSION_PROMPT='''According to the user query, expand and solidify the query into a step by step detailed instruction (or comment) on how to write python code to fulfill the user query's requirements. Import the appropriate libraries. Pinpoint the correct library functions to call and set each parameter in every function call accordingly.'''

In [56]:
@retry(wait=wait_random_exponential(min=0.02, max=1), stop=(stop_after_delay(3) | stop_after_attempt(30)))
async def _call_openai_api(system_prompt, user_content):
    try:
        response = await client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ],
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API call failed with error: {e}. Retrying...")
        raise e

In [57]:
expansion_prompt = template_expansion.format(
    ori_query=query_list,
    data_description=data_description
    )
count_gpt4o_tokens(expansion_prompt)
response_text = await _call_openai_api(system_prompt_expansion, expansion_prompt)

Token count: 533


In [58]:
count_gpt4o_tokens(response_text)

Token count: 827


827

In [59]:
match = re.search(r'\[.*\]', response_text, flags=re.DOTALL)
if match:
    generated_query = match.group().strip()
    try:
        parsed_list = ast.literal_eval(generated_query)
        # print(parsed_list)
        if isinstance(parsed_list, list) and parsed_list:
            # return parsed_list
            pass
        else:
            pass
    except (ValueError, SyntaxError) as e:
        # print(e)
        pass
else:
    print("No match found")
    pass

In [60]:
total_input = 0
total_output = 0

code_list = []

for idx, p_list in enumerate(parsed_list):
    code_prompt = template_codegen.format(
        query=query_list,
        data_description=data_description,
    )
    print(f"{idx+1} 번째 Query Extension")
    input_token = count_gpt4o_tokens(code_prompt)
    system_token = count_gpt4o_tokens(system_prompt_codegen)

    response_text = await _call_openai_api(system_prompt_codegen, code_prompt)
    output_token = count_gpt4o_tokens(response_text)

    total_input += input_token
    total_input += system_token
    
    total_output += output_token

    match = re.search(r"```python(.*?)```", response_text, flags=re.DOTALL)
    if match:
        code_content = match.group(1).strip()
        code_list.append(code_content)
    else:
        pass

print(f'최종 Input Token {total_input} / Output Token {total_output}')

1 번째 Query Extension
Token count: 346
Token count: 62
Token count: 176
2 번째 Query Extension
Token count: 346
Token count: 62
Token count: 176
3 번째 Query Extension
Token count: 346
Token count: 62
Token count: 176
최종 Input Token 1224 / Output Token 528


In [61]:
def code_to_image(code, img_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    exec_globals = {"plt": plt, "np": np, "sns" : sns}  # Define the global context for the code execution
    exec_locals = {}  # Local context for the code execution
    code = code.replace("plt.show()", f"plt.savefig('{img_path}')\nplt.close('all')")  # Replace plt.show() with savefig to save the image
    try:
        exec(code, exec_globals, exec_locals)  # Execute the code
        return True, 'No Error'  # Return True if the code executed successfully
    except Exception as e:
        return False, f'''There are some errors in the code you gave:
{str(e)}
please correct the errors.'''
    
# Function to encode the image to base64 format
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [62]:
def add_idx(path, idx):
    parts = path.rsplit('.', 1)  
    if len(parts) == 2:
        return ".".join([f"{parts[0]}_path{idx}", parts[1]]) 
    return path  

new_img_path_list = [add_idx(img_save_path, idx) for idx, _ in enumerate(code_list)]             
new_img_path_list

['./40_path0.png', './40_path1.png', './40_path2.png']

In [63]:
total_input = 0
total_output = 0

generated_feedback_list = []

for idx, (code, img_path) in enumerate(zip(code_list, new_img_path_list)):
    print(f"{idx+1} 번째 Code Feedback")

    # Save the plot as an image using the provided code
    success, error_message = code_to_image(code, img_path)

    if not success:
        base64_image = error_message  # Return an error message if image saving fails
    else:
        # Encode the image to base64 for sending as a message
        base64_image = encode_image(img_path)
    
    # Prepare the messages for GPT-4o, including the system prompt, user prompt with code and query, and the image
    messages = [
        {
            "role": "system",
            "content": '''
                Given a piece of code, a user query, and an image of the current plot, 
                please determine whether the plot has faithfully followed the user query. 
                
                Your task is to provide instruction to make sure the plot has strictly 
                completed the requirements of the query. Please output a detailed step by step 
                instruction on how to use python code to enhance the plot. If the plot image 
                is missing, check the error message that has occurred in the code. 
                
                Provide clear, essential instructions to modify the code based on the analysis, 
                focusing only on the discrepancies between the plot and the user query. 
                Avoid unnecessary feedback or suggestions. Do not output the final modified code, 
                only the instructions.'''
        },
        {
            "role": "user",
            "content" : f'''
                Here is the code: [Code]:
                    """
                    {code}
                    """

                    Here is the user query: [Query]:
                    """
                    {query_list}
                    """
                Carefully analyze the provided Python code, user query, and the plot image (if available) 
                to evaluate if the generated plot meets the user query requirements. If the plot image is missing,
                check the error message that has occurred in the code. 
                
                Compare the plot with the user query requirements, highlight discrepancies, and provide clear, 
                essential instructions to modify the code accordingly.

                Additionally, suggest improvements for better visualization, focusing on clarity, readability, 
                and alignment with the user's objectives.
                
                Provide step-by-step instructions for code modification based on the analysis, focusing only on necessary 
                corrections. Do not provide the final modified code, only the instructions for fixing the discrepancies.
                '''
        }
    ]
    input_token = count_gpt4o_tokens(messages[0]['content']) + count_gpt4o_tokens(messages[-1]['content'])
    messages[-1]["content"] += f"\n\n![plot](data:image/png;base64,{base64_image})"
    
    feedback = await _call_openai_api(messages[0]['content'], messages[-1]['content'])
    output_token = count_gpt4o_tokens(feedback)

    total_input += input_token
    total_output += output_token

    generated_feedback_list.append(feedback)

    print('Feedback is done')
    
print(f'최종 Input Token {total_input} / Output Token {total_output}')

Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


1 번째 Code Feedback
Token count: 152
Token count: 556


Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


Token count: 402
Feedback is done
2 번째 Code Feedback
Token count: 152
Token count: 556


Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


Token count: 494
Feedback is done
3 번째 Code Feedback
Token count: 152
Token count: 556
Token count: 465
Feedback is done
최종 Input Token 2124 / Output Token 1361


In [64]:
code_with_feedback = ""
for i in range(len(code_list)):
    code_with_feedback += f"------\ncode{i+1}:\n{code_list[i]}\nfeedback{i+1}:\n{generated_feedback_list[i]}\n"

In [65]:
final_code_prompt = template_aggregation.format(
    ori_query=query_list,
    data_description=data_description,
    code_for_aggregation=code_with_feedback
)
user_token = count_gpt4o_tokens(final_code_prompt)
system_token = count_gpt4o_tokens(system_prompt_codegen)
print(f"Fianl_code_prompt Input Token: {user_token+system_token}")

Token count: 2473
Token count: 62
Fianl_code_prompt Input Token: 2535


In [66]:
response_text = await _call_openai_api(system_prompt_codegen, final_code_prompt)
count_gpt4o_tokens(response_text)

match = re.search(r"```python(.*?)```", response_text, flags=re.DOTALL)
if match:
    code = match.group(1).strip()
    # return code, extend_query_list, generated_code_list, prompt
else:
    pass
    # return None, extend_query_list, generated_code_list, prompt

Token count: 449


In [69]:
# 평균 계산 함수
def calculate_averages(experiments):
    averages = {}
    for stage, data in experiments.items():
        avg_input = sum(data["input_tokens"]) / len(data["input_tokens"])
        avg_output = sum(data["output_tokens"]) / len(data["output_tokens"])
        averages[stage] = {"avg_input_tokens": avg_input, "avg_output_tokens": avg_output}
    return averages

# 새로운 실험 데이터
new_experiments = {
    "Query Expansion": {
        "input_tokens": [571, 683, 458, 492, 533],
        "output_tokens": [719, 1000, 662, 668, 827],
    },
    "Code Generation": {
        "input_tokens": [1335, 1671, 996, 1098, 1224],
        "output_tokens": [1227, 1169, 780, 1262, 528],
    },
    "Visual Feedback": {
        "input_tokens": [2934, 3212, 2148, 2732, 2124],
        "output_tokens": [1554, 1726, 1628, 1571, 1361],
    },
    "Final Code Generation": {
        "input_tokens": [3464, 3690, 2978, 3437, 2535],
        "output_tokens": [701, 657, 633, 884, 449],
    },
}

# 평균 계산 실행
new_average_tokens = calculate_averages(new_experiments)
new_average_tokens


{'Query Expansion': {'avg_input_tokens': 547.4, 'avg_output_tokens': 775.2},
 'Code Generation': {'avg_input_tokens': 1264.8, 'avg_output_tokens': 993.2},
 'Visual Feedback': {'avg_input_tokens': 2630.0, 'avg_output_tokens': 1568.0},
 'Final Code Generation': {'avg_input_tokens': 3220.8,
  'avg_output_tokens': 664.8}}